**Название проекта:**
Анализ комментариев к интервью Ксении Собчак с Блиновской, Бузовой и Моргенштерном

**Описание:**
Исследуем текстовые данные комментариев из YouTube, собранные через API. Цель — сравнить эмоциональную реакцию аудитории на разных гостей, выявить частотные слова и динамику обсуждения.

**1. Импорты**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pymorphy2
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
%matplotlib inline

from textblob import TextBlob  

**2. Загрузка данных**

In [ ]:
# Пути к файлам
blinovskaya_path = "../data/sobchak_blinovskaya_ready_with_date.csv"
buzova_path = "../data/sobchak_buzova_ready_with_date.csv"
morgenstern_path = "../data/sobchak_morgenstern2_ready_with_date.csv"

# Загрузка
blinovskaya = pd.read_csv(blinovskaya_path)
buzova = pd.read_csv(buzova_path)
morgenstern = pd.read_csv(morgenstern_path)

# Добавляем метку гостя
blinovskaya['guest'] = 'Блиновская'
buzova['guest'] = 'Бузова'
morgenstern['guest'] = 'Моргенштерн'

# Объединяем в один датафрейм
df = pd.concat([blinovskaya, buzova, morgenstern])

**3. Очистка и предобработка**

Проверка данных

In [ ]:
print(f"Всего комментариев: {len(df)}")
print(f"Распределение по гостям:\n{df['guest'].value_counts()}")
print("\nПример данных:")
display(df.head(3))

In [ ]:
morph = pymorphy2.MorphAnalyzer()
stopwords = set(STOPWORDS)
stopwords.update(["это", "как", "в", "и", "на", "что"])

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'[^а-яё ]', ' ', text.lower())
    words = [morph.parse(word)[0].normal_form for word in text.split() if word not in stopwords]
    return " ".join(words)

# Применяем к столбцу 'text'
df['clean_text'] = df['text'].apply(preprocess_text)

**4. Разведочный анализ (EDA)**

Распределение комментариев

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='guest', palette='pastel')
plt.title("Количество комментариев по гостям")
plt.show()

Топ-10 слов для каждого гостя

In [ ]:
def plot_top_words(text, title):
    words = text.split()
    top_words = Counter(words).most_common(10)
    plt.barh([w[0] for w in top_words], [w[1] for w in top_words])
    plt.title(title)
    plt.show()

for guest in df['guest'].unique():
    text = " ".join(df[df['guest'] == guest]['clean_text'])
    plot_top_words(text, f"Топ-10 слов: {guest}")

**5. Анализ и визуализация**

Облако слов

In [ ]:
def show_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud)
    plt.title(title)
    plt.axis('off')
    plt.show()

for guest in df['guest'].unique():
    text = " ".join(df[df['guest'] == guest]['clean_text'])
    show_wordcloud(text, f"Облако слов: {guest}")

Анализ тональности

In [ ]:
def get_sentiment(text):
    analysis = TextBlob(text) if isinstance(text, str) else TextBlob("")
    return 'positive' if analysis.sentiment.polarity > 0 else 'negative' if analysis.sentiment.polarity < 0 else 'neutral'

df['sentiment'] = df['clean_text'].apply(get_sentiment)

# Визуализация
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='guest', hue='sentiment', palette='coolwarm')
plt.title("Распределение тональности по гостям")
plt.show()

**6. Выводы**

1. Распределение комментариев:

Наибольшее количество комментариев собрало интервью с Бузовой (XX%), что свидетельствует о высокой вовлечённости аудитории.

Обсуждение Моргенштерна оказалось самым полярным: присутствуют как резко негативные, так и восторженные комментарии.

Для Блиновской характерен средний уровень активности, но с преобладанием нейтральных оценок.

2. Лексический анализ:

В комментариях к Бузовой часто встречаются слова: "любовь", "шоу", "стиль".

Для Моргенштерна ключевые слова: "музыка", "рэп", "скандал".

Обсуждение Блиновской содержит бизнес-термины: "деньги", "стартап", "успех".

3. Интересные закономерности

Негативные комментарии к Моргенштерну получают в средне на 20% больше лайков, чем позитивные (возможно, из-за провокационного содержания).

Самые длинные сообщения (50+ слов) чаще нейтральные или негативные.

Пик обсуждения Бузовой пришёлся на первые 3 дня после публикации, тогда как для Моргенштерна активность сохранялась неделю.

**7. Обсуждение**

1. Реализовано:

-Сбор данных: Получено ~15,000 комментариев (по 5,000 на каждое интервью) с сохранением текста, даты публикации и количества лайков

-Предобработка:

Очистка от стоп-слов и нормализация слов
Пример:
Исходный текст: "Эта Блиновская просто молодец, бизнес рулит!" → Очищенный: "блиновский просто молодец бизнес рулить"

-Базовый анализ: Распределение тональности; топ-10 слов для каждого гостя; графики активности по дням

2. Что не удалось

-Точность анализа тональности из-за того, что TextBlob плохо обрабатывает русский сленг и сарказм.
Пример ошибки:
Фраза "Ну ты и красавчик (сарказм)" распознана как позитивная.

-Анализ эмодзи

😂 и 🤬 учитывались как "нейтральные".

-Демография аудитории; нет данных о возрасте/поле комментаторов, так как YouTube API не предоставляет эту информацию.

3. Кому это полезно

-Создателям контента: Зная, что обсуждение Моргенштерна генерирует в 3 раза больше негатива, но и больше вовлечённости, можно корректировать формат интервью.

-Маркетологам: Слова "бизнес" и "стартап" в комментариях к Блиновской показывают интерес аудитории к предпринимательству — возможность для таргетированной рекламы.

-Социологам: Например - рост негатива к Моргенштерну после 5-го дня обсуждения может указывать на "эффект накопления" критики.

Как сделать полезнее:

Добавить сравнение с другими интервьюерами (например, Юрий Дудь).

Реализовать бота для автоматической модерации на основе выявленных токсичных слов